# Airbyte API Testing for adding sources

----------------------------------------------------------------

This notebook's aim is to playaround and see how airbyte api works and how to make a python script to add sources.

In [1]:
# import statments
import requests
import os
from dotenv import load_dotenv, find_dotenv

In [2]:
# airbyte api base url so that we don't keep typing it everytime

AIRBYTE_PUBLIC_API = "http://localhost:8000/api/public/v1/"
AIRBYTE_API = "http://localhost:8000/api/v1/"

In [24]:
# to access local environmental variables

load_dotenv(find_dotenv(), override=True)

API_KEY = os.getenv('BLINKMETRICS_OPENWEATHER_API_KEY')
CLIENT_ID = os.getenv('CLIENT_ID')
CLIENT_SECERT = os.getenv('CLIENT_SECRET')
WORK_SPACE = os.getenv('WORK_SPACE_ID')
PASSWORD = os.getenv('DB_PASS')

Let us first test the connection with the API to see if it is accessible

In [4]:
# testing the connection to the API:

health_url = AIRBYTE_PUBLIC_API + '/health'

health_response = requests.get(health_url)
health_response.status_code

200

In [5]:
health_response.text # seeing the response to make sure the API connection is up

'Successful operation'

The API is accessible!

Now to get the token so we can access everything else in the API:

In [6]:
# getting the Token for authorization:

url_token = AIRBYTE_PUBLIC_API + "/applications/token"

payload = {
    "client_id": CLIENT_ID,
    "client_secret": CLIENT_SECERT,
}

headers = {
    "accept": "application/json",
    "content-type": "application/json"
}

token_response = requests.post(url_token, json=payload, headers=headers)

TOKEN = token_response.json()['access_token'] # storing the token in a constant

since we now got the token we can proceed with testing how to create a source :D

## Create a source:

**1. get the source definition:**

In [7]:
source_def_url = AIRBYTE_API + "/source_definitions/list" 

headers = {
    "accept": "application/json",
    "Content-Type" : "application/json",
    "authorization" : "Bearer " + TOKEN
} 

source_def_response = requests.post(source_def_url, headers=headers)
source_def_response.status_code

200

In [8]:
# getting the source def of OpenWeather so we can use it in the create source request 

for source_def in source_def_response.json()['sourceDefinitions']:
        if 'openweather' in source_def['name'].lower():
             SOURCE_DEF = source_def['sourceDefinitionId']
             
SOURCE_DEF

'561d7787-b45e-4f3b-af58-0163c3ba9d5a'

**2. create the source:**

In [9]:
url = AIRBYTE_API + "/sources/create"

payload = {
    "sourceDefinitionId": SOURCE_DEF,
    "workspaceId": WORK_SPACE,
    
    "connectionConfiguration": {
        "appid": API_KEY,
        "lat": '33.8959203',
        "lon": '35.47843'
    },

    "name": f"OpenWeather - Beirut - test"
}

headers = {
    "accept": "application/json",
    "Content-Type" : "application/json",
    "authorization" : "Bearer " + TOKEN
} 

response = requests.post(url, json=payload, headers=headers)
response.json()

{'sourceDefinitionId': '561d7787-b45e-4f3b-af58-0163c3ba9d5a',
 'sourceId': '5201d6c0-05db-4627-8134-b431a6a0b4d1',
 'workspaceId': 'b0841efb-2af8-410b-afe0-40a9837d3912',
 'connectionConfiguration': {'lat': '33.8959203',
  'lon': '35.47843',
  'appid': '**********'},
 'name': 'OpenWeather - Beirut - test',
 'sourceName': 'Openweather',
 'icon': 'https://connectors.airbyte.com/files/metadata/airbyte/source-openweather/latest/icon.svg',
 'isVersionOverrideApplied': False,
 'supportState': 'supported'}

seems like our test was successful, let us see if the source appears in the UI

<img src="..\assets\source_test_success.jpg" width="750"/>

the source did appear in the UI :D

now to try creating a destination 

## Create a Destination:

In [11]:
dest_def_url = AIRBYTE_API + "destination_definitions/list" 

headers = {
    "accept": "application/json",
    "Content-Type" : "application/json",
    "authorization" : "Bearer " + TOKEN
} 

dest_def_response = requests.post(dest_def_url, headers=headers)
dest_def_response.status_code

200

In [13]:
for destination_def in dest_def_response.json()['destinationDefinitions']:
    if 'MySQL'.lower() in destination_def['name'].lower():
        DEST_DEF = destination_def['destinationDefinitionId']

In [23]:

url = AIRBYTE_API + "/destinations/create"

payload = { 
    "destinationDefinitionId": DEST_DEF,
    "workspaceId": WORK_SPACE,

    "connectionConfiguration": {
        "destinationType": "mysql",
        "host": "host.docker.internal",
        "port": 3306,
        "database": "blinkmetrics_project_stage",
        "username": "root",
        "password": PASSWORD
    },

    "name": "MySQL - test"}

headers = {
    "accept": "application/json",
    "content-type": "application/json",
    "authorization": "Bearer " + TOKEN
}

response = requests.post(url, json=payload, headers=headers)

response.text

'{"destinationDefinitionId":"ca81ee7c-3163-4246-af40-094cc31e5e42","destinationId":"da8ae1aa-30e0-4487-87d8-f5c05b84421f","workspaceId":"b0841efb-2af8-410b-afe0-40a9837d3912","connectionConfiguration":{"host":"host.docker.internal","port":3306,"database":"blinkmetrics_project_stage","password":"**********","username":"root","destinationType":"mysql"},"name":"MySQL - test","destinationName":"MySQL","icon":"https://connectors.airbyte.com/files/metadata/airbyte/destination-mysql/latest/icon.svg","isVersionOverrideApplied":false,"supportState":"supported"}'

<img src="..\assets\destination_test_success.jpg" width="750"/>

This was a success too :D!